# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## (**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

我们现在可以将存储在文件中的数据读回内存。


In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以[**存储一个张量列表，然后把它们读回内存。**]


In [3]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。


In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）确实有用，
但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数。
让我们从熟悉的多层感知机开始尝试一下。


In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]


In [6]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)


In [7]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。


In [8]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。

## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？<br>
    1. **恢复和继续训练**：存储模型参数可以帮助您保存模型的状态，以便稍后重新加载并继续训练。这对于长时间运行的训练任务非常有用，因为它允许您定期保存模型的中间状态，以防止训练过程中出现意外情况。

    2. **共享和复用**：您可以将经过训练的模型参数与他人共享，或者在不同的项目中重复使用。这可以节省时间和计算资源，特别是当您在多个项目中使用相似的模型时。

    3. **对抗性训练**：在对抗性机器学习中，攻击者可能尝试攻击您的模型。通过保存模型的参数，您可以更容易地检测到模型受到的攻击，然后采取适当的反对策略。

    4. **验证和解释性**：将模型参数存储为可解释的格式可以帮助您验证模型的正确性，并使模型的内部结构对您更加透明和可理解。这对于监管机构或法律要求的透明性要求非常有用。

    5. **模型复制**：如果您需要在多个环境中使用相同的模型，您可以将模型参数复制到不同的设备或云服务器，而无需重新训练整个模型。

    总之，存储模型参数是一种良好的实践，它有助于提高模型的可维护性、可重用性和可解释性，同时增加模型的稳健性和安全性。
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？


In [21]:
import torch
import torch.nn as nn

class Net1(nn.Module):

    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.ReLU(),
        )
    
    def forward(self, X):
        return self.layers(X)


class Net2(nn.Module):

    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.ReLU(),
        )
    def forward(self, X):
        return self.layers(X)
    

x = torch.rand(2, 20)
net1 = Net1(20, 10)
net2 = Net2(40, 10)
net1(x)

for layer_name, parameter in net2.layers[::].named_parameters():
    print(f'{layer_name = }', parameter.shape)

for i, layer in enumerate(net1.layers[4:6]):
    net2.layers.insert(i + 4, layer)

for layer_name, parameter in net2.layers[::].named_parameters():
    print(f'{layer_name = }', parameter.shape)

x = torch.rand(2, 40)
net2(x)

layer_name = '0.weight' torch.Size([256, 40])
layer_name = '0.bias' torch.Size([256])
layer_name = '2.weight' torch.Size([512, 256])
layer_name = '2.bias' torch.Size([512])
layer_name = '4.weight' torch.Size([10, 512])
layer_name = '4.bias' torch.Size([10])
layer_name = '0.weight' torch.Size([256, 40])
layer_name = '0.bias' torch.Size([256])
layer_name = '2.weight' torch.Size([512, 256])
layer_name = '2.bias' torch.Size([512])
layer_name = '4.weight' torch.Size([512, 512])
layer_name = '4.bias' torch.Size([512])
layer_name = '6.weight' torch.Size([10, 512])
layer_name = '6.bias' torch.Size([10])


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0385, 0.0012, 0.0567, 0.0004, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0429, 0.0018, 0.0595, 0.0035, 0.0000,
         0.0022]], grad_fn=<ReluBackward0>)

[Discussions](https://discuss.d2l.ai/t/1839)


#### 如何同时保存网络架构和参数？需要对架构加上什么限制？

要同时保存神经网络的架构和参数，您可以使用PyTorch的内置功能。通常情况下，您可以将网络的架构和参数一起保存到一个文件中，然后在需要时加载它们。

以下是一种通用的保存和加载神经网络架构和参数的方法：

1. **保存模型**：

    ```python
    torch.save(model, 'model.pth')
    ```

    这将保存整个模型，包括架构和参数，到名为'model.pth'的文件中。

2. **加载模型**：

    ```python
    model = torch.load('model.pth')
    ```

    这将加载之前保存的模型，包括架构和参数。

注意事项：

- 要成功加载模型，您需要确保定义模型的代码与保存时完全相同。这意味着模型类的定义、架构和参数的顺序等都必须保持一致。
- 如果需要更多的灵活性，例如加载模型到不同的架构，您可以保存和加载模型的状态字典（state_dict）而不是整个模型。

以下是保存和加载模型状态字典的示例：

1. **保存模型状态字典**：

    ```python
    torch.save(model.state_dict(), 'model_state.pth')
    ```

    这将保存模型的状态字典到名为'model_state.pth'的文件中。

2. **加载模型状态字典**：

    ```python
    # 首先创建模型，确保模型结构与保存时一致
    model = YourModelClass()
    
    # 加载模型状态字典
    model.load_state_dict(torch.load('model_state.pth'))
    ```

这种方法允许更大的灵活性，因为您可以将状态字典加载到与保存时不同的模型架构中，只要架构兼容。确保任何改变都不会破坏参数的对应性。

请注意，保存和加载模型时需要谨慎，以确保一致性，以免导致错误。